In [3]:
import tkinter as tk
from tkinter import filedialog
from tkinter import scrolledtext
import re

In [15]:
# Define token types
TOKEN_TYPES = [
    ('IF', r'if'),
    ('ELSE', r'else'),
    ('WHILE', r'while'),
    ('FOR', r'for'),
    ('ID', r'[a-zA-Z_][a-zA-Z0-9_]*'),
    ('INT', r'\d+'),
    ('FLOAT', r'\d+\.\d+'),
    ('STRING', r'\".*?\"'),
    ('PLUS', r'\+'),
    ('MINUS', r'-'),
    ('TIMES', r'\*'),
    ('DIVIDE', r'/'),
    ('LPAREN', r'\('),
    ('RPAREN', r'\)'),
    ('LBRACE', r'\{'),
    ('RBRACE', r'\}'),
    ('ASSIGN', r'='),
    ('SEMICOLON', r';'),
    ('NEWLINE', r'\n'),
    ('WS', r'\s+'),  # Include whitespace token type
]

# Concatenate all token regexes into one string for lex to use
TOKEN_REGEX = '|'.join('(?P<%s>%s)' % pair for pair in TOKEN_TYPES)

# Tokenize input string
def tokenize(input_string):
    return [match.group(0) for match in re.finditer(TOKEN_REGEX, input_string)]

# Tokenize input file
def tokenize_file(file_path):
    with open(file_path, 'r') as file:
        input_string = file.read()
    return tokenize(input_string)

def parse(tokens):
    i = 0
    statements = []
    while i < len(tokens):
        if tokens[i] == 'if':
            parse_result, i = parse_if(tokens, i)
            statements.append(parse_result)
        elif tokens[i] in ['int', 'float', 'string']:
            parse_result, i = parse_variable_declaration(tokens, i)
            statements.append(parse_result)
        elif tokens[i] == 'while':
            parse_result, i = parse_while(tokens, i)
            statements.append(parse_result)
        elif tokens[i] in {'\n', ' '}:
            i += 1  # Skip whitespace characters
            continue
        else:
            print("No parse result. Unexpected token:", tokens[i])
            return None
    return statements

def parse_if(tokens, i):
    if_index = tokens.index('{', i)
    condition_tokens = tokens[i + 2:if_index - 1]  # Adjusted range to exclude '(' and ')'
    
    # Find the index of '}' token for the if block
    close_brace_index = if_index
    brace_count = 1
    while brace_count != 0:
        close_brace_index += 1
        if tokens[close_brace_index] == '{':
            brace_count += 1
        elif tokens[close_brace_index] == '}':
            brace_count -= 1
    
    if_block_tokens = tokens[if_index + 1:close_brace_index]  # Extract tokens inside the if block
    
    # Find the index of '{' token for the else block
    else_index = close_brace_index + 1
    while tokens[else_index] != '{':
        else_index += 1
    
    # Find the index of '}' token for the else block
    close_else_brace_index = else_index
    brace_count = 1
    while brace_count != 0:
        close_else_brace_index += 1
        if tokens[close_else_brace_index] == '{':
            brace_count += 1
        elif tokens[close_else_brace_index] == '}':
            brace_count -= 1
    
    else_block_tokens = tokens[else_index + 1:close_else_brace_index]  # Extract tokens inside the else block
    
    # Create parse result for if-else statement
    parse_result = ('if_else', parse_expression(condition_tokens), parse_block(if_block_tokens), parse_block(else_block_tokens))
    
    return parse_result, close_else_brace_index + 1  # Move index past the closing '}'

def parse_variable_declaration(tokens, i):
    end_index = tokens.index(';', i)
    parse_result = ('variable_declaration', tokens[i:end_index + 1])
    return parse_result, end_index + 1  # Move index past the ';'

def parse_while(tokens, i):
    open_brace_index = i + 1
    while tokens[open_brace_index] != '{':
        open_brace_index += 1

    close_brace_index = open_brace_index
    brace_count = 1
    while brace_count != 0:
        close_brace_index += 1
        if tokens[close_brace_index] == '{':
            brace_count += 1
        elif tokens[close_brace_index] == '}':
            brace_count -= 1

    condition_tokens = tokens[i + 2:open_brace_index - 1]  # Adjusted range to exclude '(' and ')'
    block_tokens = tokens[open_brace_index + 1:close_brace_index]  # Extract tokens inside the block
    parse_result = ('while', parse_expression(condition_tokens), parse_block(block_tokens))
    return parse_result, close_brace_index + 1  # Move index past the closing '}'

def parse_expression(expression_tokens):
    # Assuming simple expressions like 'x > 0'
    return ('expression', expression_tokens)

def parse_block(block_tokens):
    statements = []
    statement = []
    for token in block_tokens:
        if token == ';':
            statements.append(parse_statement(statement))
            statement = []
        else:
            statement.append(token)
    return ('block', statements)

def parse_statement(statement_tokens):
    # Assuming simple assignments like 'x = x + 1'
    return ('statement', statement_tokens)


class ParserApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Welcome to C-like Compiler")
        
        # Welcome message and description
        self.welcome_label = tk.Label(master, text="Welcome to C-like Compiler", font=("Helvetica", 16, "bold"))
        self.welcome_label.pack(pady=10)
        
        self.description_label = tk.Label(master, text="This tool parses C-like code and checks for syntax errors.", font=("Helvetica", 12))
        self.description_label.pack(pady=5)
        
        # Text area for code input
        self.text_area = scrolledtext.ScrolledText(master, width=50, height=20)
        self.text_area.pack(expand=True, fill='both')
        
        # Buttons for file handling and functionality
        self.btn_open = tk.Button(master, text="Open File", command=self.open_file)
        self.btn_open.pack()
        
        self.btn_parse = tk.Button(master, text="Parse", command=self.parse_file)
        self.btn_parse.pack()
        
        self.btn_check_syntax = tk.Button(master, text="Check Syntax", command=self.check_syntax)
        self.btn_check_syntax.pack()
        
        # Error logger
        self.error_logger = scrolledtext.ScrolledText(master, width=50, height=5)
        self.error_logger.pack()

    def open_file(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            with open(file_path, 'r') as file:
                content = file.read()
                self.text_area.insert('1.0', content)

    def parse_file(self):
        content = self.text_area.get('1.0', 'end-1c')
        # Check if the user has written a test case or uploaded a file
        if content.strip() == "":
            self.error_logger.insert('end', "Error: No input provided.\n")
            return
        tokens = tokenize(content)
        parse_tree = parse(tokens)
        if tokens:
            print("\nTokens:", tokens)
        if parse_tree:
            print("\n\nParse tree:", parse_tree)
        else:
            print("Parsing failed.")

    def check_syntax(self):
        content = self.text_area.get('1.0', 'end-1c')
        if content.strip() == "":
            self.error_logger.insert('end', "Error: No input provided.\n")
            return
        tokens = tokenize(content)
        parse_tree = parse(tokens)
        if not parse_tree:
            self.error_logger.insert('end', "Syntax Error: Parsing failed.\n")
        else:
            self.error_logger.insert('end', "Syntax Check Passed.\n")

def main():
    root = tk.Tk()
    app = ParserApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()